# File for generating dataset for and training the query embedding Model
 - Lots of dicts

In [1]:
%matplotlib inline

import gensim.downloader as gensim_api
corpus = gensim_api.load('text8')
from torch.distributions import categorical
from torch.utils import data
import pandas as pd
from thefuzz import fuzz
from thefuzz import process
from gensim.models.word2vec import Word2Vec
import json, codecs
import tensorflow as tf
from torch import nn
import numpy as np
import collections
import torch
import random
from nltk import tokenize


/home/theo/anaconda3/envs/prawn/lib/python3.7/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [23]:
def gen_dict(df,keys,values):
    _list = list(zip(df[keys],df[values]))
    c = collections.defaultdict(list)
    for a,b in _list:
        c[a].extend([b])

    for key in list(c.keys()):
        if len(c[key]) < 2:
            del c[key]
    print(len(c),'c length')
    return c


## Load Results

In [331]:
cleaned_bt = pd.read_csv('./cleaned_books_and_tags')
with open('dict.json') as json_file:
    correct_word_dict = json.load(json_file)
    
with open('book_to_int.json') as json_file:
    book_to_int = json.load(json_file)
    
with open('int_to_book.json') as json_file:
    int_to_book = json.load(json_file)
    
with open('book_id_to_correct_title.json') as json_file:
    book_id_to_correct_title = json.load(json_file)
    
with open('incorrect_title_to_book_id.json') as json_file:
    incorrect_title_to_book_id = json.load(json_file)
    
with open('int_to_weight.json') as json_file:
    temp = json.load(json_file)
int_to_weight = {int(k):np.asarray(v) for k,v in temp.items()}
    
corpus = gensim_api.load('text8')
model = Word2Vec(corpus)

#a bit hacky but usefull later
model.wv.add_vector("__null__",[0 for i in range(100)])

/home/theo/anaconda3/envs/prawn/lib/python3.7/site-packages/gensim/models/keyedvectors.py:481: UserWarning: Adding single vectors to a KeyedVectors which grows by one each time can be costly. Consider adding in batches or preallocating to the required size.
  UserWarning)


71290

In [332]:
t = pd.read_csv( 'tags.csv' )
b = pd.read_csv( 'books.csv' )
bt = pd.read_csv( 'book_tags.csv')
r = pd.read_csv( 'ratings.csv' )
r = r.merge(b[['book_id','original_title']], on = 'book_id')
bt = bt.merge( t, on = 'tag_id' )
bt = bt.merge( b[[ 'goodreads_book_id', 'title','book_id']], on = 'goodreads_book_id' )



In [333]:
cleaned_bt = cleaned_bt.merge( b[[ 'goodreads_book_id']], on = 'goodreads_book_id' )
cleaned_bt

,Unnamed: 0,goodreads_book_id,tag_id,count,tag_name,title
0,0,1,30574,167697,to-read,Harry Potter and the Half-Blood Prince (Harry ...
1,1,1,11305,37174,fantasy,Harry Potter and the Half-Blood Prince (Harry ...
2,2,1,11557,34173,favorites,Harry Potter and the Half-Blood Prince (Harry ...
3,3,1,8717,12986,currently-reading,Harry Potter and the Half-Blood Prince (Harry ...
4,4,1,33114,12716,young-adult,Harry Potter and the Half-Blood Prince (Harry ...
...,...,...,...,...,...,...
864073,999907,675614,33057,7,ya-witches,"Book of Shadows (Sweep, #1)"
864074,999908,675614,32624,6,witches-and-magic,"Book of Shadows (Sweep, #1)"
864075,999909,675614,32612,6,witch-books,"Book of Shadows (Sweep, #1)"
864076,999910,675614,32625,5,witches-and-warlocks,"Book of Shadows (Sweep, #1)"


In [334]:
book_id_to_correct_title = dict(zip(r.book_id,r.original_title))
incorrect_title_to_book_id = dict(zip(bt.title, bt.book_id))

book_id_to_correct_title_r = dict(zip(r.original_title, r.book_id))
for key,value in book_id_to_correct_title_r.items():
    if key not in incorrect_title_to_book_id.keys():
        incorrect_title_to_book_id[key] = value

cleaned_bt.title = cleaned_bt.title.apply(lambda x: book_id_to_correct_title[incorrect_title_to_book_id[x]]) #fix naming
r.original_title = r.original_title.apply(lambda x: book_id_to_correct_title[incorrect_title_to_book_id[x]])


In [335]:
cleaned_bt_by_count = cleaned_bt.sort_values('count', ascending=False)

In [336]:
good_tags = (cleaned_bt_by_count['tag_name'][cleaned_bt['count'] < 2000].unique())

In [355]:
cleaned_bt.dropna(inplace=True)
title_to_tag = gen_dict(cleaned_bt,'title','tag_name')
for title,tags in title_to_tag.items():
    new_tags = [tag for tag in tags if tag in good_tags]
    title_to_tag[title] = new_tags

9255 c length


In [357]:
def queryFromTags(tag_list, q_length, correct_word_dict, 
                  w2v=model, 
                  distribution_over_length = None, sample_similar_prob = .3):
    '''
    function for generating a synthetic query.
    odds_per_count is the distribution over various sentence lengthes to sample from.
    distribution_over_vocab is some distribution over tags to be sampled from.
    '''
    total_vocab = w2v.wv.key_to_index.keys()
    #max_q_length = min(len(tag_list),max_q_length)
    
    
    if distribution_over_length == None:
        distribution_over_length = categorical.Categorical(probs = torch.ones(q_length)/q_length)

    query_words = []
    tag_list = random.sample(tag_list,30)
    single_tags = []
    for tag in tag_list:
        single_tags.extend(tag.split('-'))
        
    single_tags = list(filter(lambda tag: len(tag)>2, single_tags))
    single_tags = list(set(single_tags))
    for i,tag in enumerate(single_tags):
        if tag in correct_word_dict.keys():
            single_tags[i] = correct_word_dict[tag][0]

    out = random.sample(single_tags,q_length)
    return out
    
    
    return random.choices(query_words,k=q_length)
    
def query_wrapper(q_length, correct_word_dict, w2v, distribution_over_length=None, sample_similar_prob =.2):
    return lambda tag_list: queryFromTags(tag_list,q_length,correct_word_dict,model)

In [358]:
class TextDataset(data.Dataset):
    '''Dataset for returning (book, text) pairs
       Somewhat overkill seeing as the database is just in memory already,
       but could rewrite __getitem__ method for reading from disc 
       and use a few workers for fast collection.
    '''
    def __init__(self, title_to_tag, query_gen, book_vectors,book_to_int,int_to_book, w2v):
        self.title_to_tag = title_to_tag       
        self.query_gen = query_gen        
        self.book_vectors = book_vectors
        
        self.w2v = w2v
        
        #self.max_seq_length = max_seq_length
        
        
    def __len__(self):
        return len(self.title_to_tag)

    def __getitem__(self, idx):
        title = list(title_to_tag.keys())[idx]
        
        tags = self.title_to_tag[title]
        tag_list = self.query_gen(tags)
#         num_tags = min(self.max_seq_length,len(tag_list))
#         num_tags = self.max_seq_length
        
        
#         if len(tag_list) >= self.max_seq_length:
#             tag_list = tag_list[:self.max_seq_length]
#         else:
#             tag_list.extend(['__null__' for _ in range(self.max_seq_length-len(tag_list))])
        [self.w2v.wv[tag] for tag in tag_list]
        return torch.tensor([self.w2v.wv[tag] for tag in tag_list]).float(), torch.tensor(self.book_vectors[book_to_int[title]]).float()
        
        tag_vectors = torch.tensor([self.w2v.wv[tag] for tag in tag_list])
        book_vector = torch.tensor(self.book_vectors[book_to_int[title]])
        
        attention_mask = torch.tensor([-1 for _ in range(num_tags)] + [0 for _ in range(self.max_seq_length-num_tags)])
        
    
        return ((tag_vectors.float(), attention_mask.float()), book_vector.float())

In [359]:
def customLoader(dataset,batch_size):
    length = len(dataset)
    indices = list(range(length))
    random.shuffle(indices)
    idx = 0
    while True:
        if idx + batch_size > length:
            idx = 0
            random.shuffle(indices)
            
        batch = [dataset[indices[i]] for i in range(idx,idx+batch_size)]
        X,Y = zip(*batch)
        
        yield torch.stack(X,dim = 0), torch.stack(Y,dim = 0)

In [442]:
class TransformerEmbed(torch.nn.Module):
    '''
    TransformerEncoder for word embeddings. Since the synthetic queries are generated with out any type of order,
    we don't use a positional encoding.
    '''
    def __init__(self, in_dim, out_dim, transformer_dim=512):
        super(TransformerEmbed, self).__init__()
        
        encoder_layer = nn.TransformerEncoderLayer(d_model=transformer_dim, nhead=8,batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=3)
        
        self.initial = nn.Linear(in_dim,transformer_dim)
        self.final = nn.Linear(transformer_dim, out_dim)
        self.float()
    def call(self, inputs):
        x = self.initial(inputs)
        x = self.transformer_encoder(x)
        x = torch.mean(x,dim=1)
        x = nn.Sigmoid()(x)
        x = self.final(x)

        return x
        

In [477]:
class TrainWrapper():
    '''
    wrapper for handling training
    '''
    def __init__(self, model, lr, dataset, batch_size):
        self.model = model
        self.optim = torch.optim.Adam(self.model.parameters(), lr = lr)
        self.batch_generator = customLoader(dataset,batch_size)
        self.model.float()
        
    def Train(self,num_epochs, num_steps_per_epoch):
        for epoch in range(num_epochs):
            for step in range(num_steps_per_epoch):
                loss = self.__train()
                print(loss)
                
    def __train(self):
        x, Y = next(self.batch_generator)
       
        self.optim.zero_grad()

        out = self.model.call(x)
        
        loss = self.loss(out,Y)
        loss.backward()
        self.optim.step()
        
        return loss.detach()
        
    def loss(self,out,labels):
        #print(labels.shape)
        return nn.MSELoss()(out,labels)
        
        
        

In [480]:
transformer = TransformerEmbed(100,80)
trainWrapper = TrainWrapper(transformer,3e-4,textData,512)

In [495]:
trainWrapper.Train(30,80)

In [491]:
def query_vector(query,model,word2vec):
    query = torch.tensor([[word2vec.wv[q] for q in query]])
    return model.call(query)
    

In [493]:
torch.save(transformer.state_dict(), './query_model')